In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


In [2]:
import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
from numpy.random import seed

from inception_model_generator import MRNet_inc_model

import tensorflow as tf


Using TensorFlow backend.


#trainning on augmented balanced data with only the skewed class augmented
20 epochs


In [0]:
label_type = 'meniscus'
exam_view = 'axial'
model, cbs = MRNet_inc_model(1,0.00001,combination=[label_type, exam_view])
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos

class_weight = {0: 1, 1: neg/pos}
if neg < pos:
  class_weight = {0: pos/neg, 1: 1}

print(class_weight)
batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l ,{0:1,1:1}, batch_size=1,model="inception",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)
test_data_gen = MRNet_data_generator(data_path, p, l,{0:1,1:1} ,batch_size=1,model="inception",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)


model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception/"+label_type+"/"+exam_view+"/weights.50.hdf5")
model.fit(x=train_data_gen, validation_data = test_data_gen,epochs=75, initial_epoch = 50,verbose=True, class_weight=class_weight,callbacks=[cbs[0]],shuffle=False, use_multiprocessing=True, workers=6)

no dropout
no dropout
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_2 (MR (None, 1)                 21804833  
Total params: 21,804,833
Trainable params: 21,770,401
Non-trainable params: 34,432
_________________________________________________________________
{0: 1, 1: 1.778688524590164}
Initializing Data Generator:
factor :  1.0
model:  inception
data type:  train
Combination:  meniscus  and  axial
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/train
Number of inputs:  1017
input size:  (299, 299)
Initializing Data Generator:
factor :  1.0
model:  inception
data type:  test
Combination:  meniscus  and  axial
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/test
Number of inputs:  113
input size:  (299, 299)
Epoch 51/75
1017/1017 [==============================] - 593s 583ms/step - loss: 0.5097 - t

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 1 could not be retrieved. It could be because a worker has died.
  UserWarning)


1017/1017 [==============================] - 553s 544ms/step - loss: 0.4608 - tp: 1240.1062 - fp: 415.2114 - tn: 2109.0679 - fn: 134.6145 - accuracy: 0.8588 - precision: 0.7489 - recall: 0.9019 - auc: 0.9306 - val_loss: 0.2036 - val_tp: 1420.8319 - val_fp: 470.9203 - val_tn: 2420.0530 - val_fn: 152.1947 - val_accuracy: 0.8604 - val_precision: 0.7511 - val_recall: 0.9033 - val_auc: 0.9313

Epoch 00054: saving model to /content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception/meniscus/axial/weights.54.hdf5
Epoch 55/75
1017/1017 [==============================] - 522s 513ms/step - loss: 0.4600 - tp: 1599.4779 - fp: 518.4622 - tn: 2740.4878 - fn: 170.5723 - accuracy: 0.8629 - precision: 0.7550 - recall: 0.9036 - auc: 0.9326 - val_loss: 2.0823 - val_tp: 1779.3097 - val_fp: 570.7522 - val_tn: 3055.5576 - val_fn: 188.3805 - val_accuracy: 0.8643 - val_precision: 0.7571 - val_recall: 0.9043 - val_auc: 0.9325

Epoch 00055: saving model to /content/gdrive/My Drive/Colab Notebooks/MRNet/tr

# **testing on validation dataset**

In [0]:
label_type = 'meniscus'
exam_view = 'axial'
model, cbs = MRNet_inc_model(1,0,combination=[label_type, exam_view])
# print(model(tf.ones((12, 24, 224, 224, 3))).shape)

model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception/"+label_type+"/"+exam_view+"/weights.45.hdf5")
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['valid'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['valid'][exam_view])

neg = total - pos
class_count_valid = {0: neg, 1: pos}

batch_size = 1
print(model.metrics_names)
valid_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_valid , batch_size=1,model="inception",data_type='valid', shuffle=False,label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)
d = model.evaluate_generator(valid_data_gen, total, verbose=1)
print(d)
model.predict_generator(valid_data_gen, 5, verbose=1)

no dropout
no dropout
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_2 (MR (None, 1)                 21804833  
Total params: 21,804,833
Trainable params: 21,770,401
Non-trainable params: 34,432
_________________________________________________________________
['loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc']
Initializing Data Generator:
factor :  1.3076923076923077
model:  inception
data type:  valid
Combination:  meniscus  and  axial
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/valid
before start
Number of inputs:  120
input size:  (299, 299)
120/120 [==============================] - 129s 1s/step
[0.2905656695365906, 21.016666412353516, 19.84166717529297, 16.08333396911621, 3.558333396911621, 0.6281647086143494, 0.5177689790725708, 0.8857750296592712, 0.6143258213996887]
5/5 [======================

array([[0.7960527 ],
       [0.8246279 ],
       [0.8647641 ],
       [0.11677624],
       [0.8015888 ]], dtype=float32)